In [ ]:
import cv2
import requests

# For Android IP Webcam
# import numpy as np
# from requests.auth import HTTPBasicAuth

from dotenv import load_dotenv
import os
load_dotenv()

from ultralytics import YOLO

from threading import Timer
import time
import datetime

import base64

user = os.getenv('USER')
password = os.getenv('PASSWORD')
url = os.getenv('URL')

ntfy_user=os.getenv('NTFY_USER')
ntfy_pass=os.getenv('NTFY_PASS')

proj_path = os.getenv('PROJ_PATH')

host = os.getenv('HOSTNAME')
topic = os.getenv('TOPIC')

model = YOLO(os.getenv('MODEL'))

suspend = os.getenv('SUSPEND')

os.environ["OPENCV_FFMPEG_CAPTURE_OPTIONS"] = "rtsp_transport;0"

In [2]:
def get_img_from_ipcam_stream(
    url:str = None,
    user:str = None,
    password:str = None
    ):
  
  if url is None:
    raise ValueError("url is Empty")
  if user is None or password is None:
    raise Warning("Username and password not set, check if your server is set to with password.")
  
  # ipcam = cv2.VideoCapture(url, cv2.CAP_FFMPEG)
  # ret, frame = ipcam.read()
  # if ret:
  #   img = cv2.imdecode(frame, cv2.IMREAD_COLOR)

  # For Android IP webcam
  img_resp = requests.get(url, auth=HTTPBasicAuth(username=user, password=password))
  img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
  img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
  
  return img

In [3]:
def extract_model_prediction(model, img) -> dict:
  result = model(img)[0]
  max_conf = 0
  for conf, cs in zip(result.boxes.conf, result.boxes.cls):
    if conf >= 0.3 and result.names[int(cs)] == 'cat' and (conf >= max_conf):
      max_conf = conf
  
  msg_dict = None
  if max_conf >= 0.8:
    msg_dict = {
      "pr": "default",
      "title": "Cat Detected!",
      "msg": f"Found cat at confedence level: {conf}",
      # "img": img,
      "tags": "tada"
    }
  elif max_conf >= 0.3:
    msg_dict = {
      "pr": "low",
      "title": "Cat Detected! Probably...",
      "msg": f"Found cat at confedence level: {conf}. This could be wrong.",
      # "img": img,
      "tags": "tada"
    }
  else:
    msg_dict = None

  return msg_dict

In [4]:
def push_ntfy(
    host:str = None,
    topic:str = None,
    msg_dict:dict= None,
    img_path = None
    ):
  auth = base64.b64encode((ntfy_user+":"+ntfy_pass).encode('UTF-8'))

  requests.post(
    f"https://{host}/{topic}",               
    data=msg_dict['msg'].encode(encoding='utf-8'),
    headers={
      "Authorization": auth,
      "Title": msg_dict['title'],
      "Priority": msg_dict['pr'],
      "Tags": msg_dict['tags']
    }
  )

  if img_path is not None:
    data = open(img_path, "rb")
    filename = img_path.split('/')[-1]
    requests.put(
      f"https://{host}/{topic}",
      data=data,
      headers={"Filename": filename}
    )

In [ ]:
def chk_crt_path(path):
  if not os.path.exists(path):
    os.mkdir(path)
  elif not os.path.isdir(path):
    raise ValueError("path given exists and is not a directory.")

In [6]:
import cv2
import threading

class RTSPStream:
  def __init__(self, rtsp_url):
    self.rtsp_url = rtsp_url
    self.cap = cv2.VideoCapture(self.rtsp_url, cv2.CAP_FFMPEG)
    self.frame = None
    self.lock = threading.Lock()
    self.running = True
    threading.Thread(target=self.update, daemon=True).start()

  def update(self):
    while self.running:
      ret, frame = self.cap.read()
      if ret:
        with self.lock:
          self.frame = frame

  def get_frame(self):
    with self.lock:
      return self.frame.copy() if self.frame is not None else None

  def stop(self):
    self.running = False
    self.cap.release()

In [ ]:
# Initialize threaded RTSP stream
stream = RTSPStream(url)

threshold = 500
activation = 0
step = 20

chk_crt_path(f"{proj_path}/saved_img")

while True:
  try:
    frame = stream.get_frame()
    filename=f"{proj_path}/saved_img/{datetime.datetime.now().strftime('%c')}.jpg"
    
    if frame is not None:
      msg_dict = extract_model_prediction(model, frame)
      
      if msg_dict is not None:
        activation += step
      else:
        activation -= (step/10)

      if activation >= threshold:
        cv2.imwrite(filename, frame)
        push_ntfy(host, topic, msg_dict, filename)

        time.sleep(SUSPEND)
        activation = 0
      
  except KeyboardInterrupt:
    break

stream.stop()

[h264 @ 0x15085a140] top block unavailable for requested intra mode
[h264 @ 0x15085a140] error while decoding MB 42 0
[h264 @ 0x15085a140] out of range intra chroma pred mode
[h264 @ 0x15085a140] error while decoding MB 14 2
[h264 @ 0x15085a140] top block unavailable for requested intra mode
[h264 @ 0x15085a140] error while decoding MB 48 0
[h264 @ 0x15085a140] mb_type 34 in P slice too large at 74 0
[h264 @ 0x15085a140] error while decoding MB 74 0
[h264 @ 0x15085a140] top block unavailable for requested intra mode
[h264 @ 0x15085a140] error while decoding MB 44 0
[h264 @ 0x15085a140] top block unavailable for requested intra mode
[h264 @ 0x15085a140] error while decoding MB 61 0
[h264 @ 0x15085a140] top block unavailable for requested intra mode
[h264 @ 0x15085a140] error while decoding MB 49 0
[h264 @ 0x15085a140] cbp too large (61) at 47 0
[h264 @ 0x15085a140] error while decoding MB 47 0
[h264 @ 0x15085a140] negative number of zero coeffs at 6 3
[h264 @ 0x15085a140] error while de

frame is not empty!



[h264 @ 0x15059ca40] top block unavailable for requested intra mode
[h264 @ 0x15059ca40] error while decoding MB 42 0
[h264 @ 0x15059a3c0] out of range intra chroma pred mode
[h264 @ 0x15059a3c0] error while decoding MB 14 2
[h264 @ 0x1505c3db0] top block unavailable for requested intra mode
[h264 @ 0x1505c3db0] error while decoding MB 48 0
[h264 @ 0x1505cc610] mb_type 34 in P slice too large at 74 0
[h264 @ 0x1505cc610] error while decoding MB 74 0
[h264 @ 0x1505d4e70] top block unavailable for requested intra mode
[h264 @ 0x1505d4e70] error while decoding MB 44 0
[h264 @ 0x1505dd6d0] top block unavailable for requested intra mode
[h264 @ 0x1505dd6d0] error while decoding MB 61 0
[h264 @ 0x1505e5f30] top block unavailable for requested intra mode
[h264 @ 0x1505e5f30] error while decoding MB 49 0
[h264 @ 0x1505ee790] cbp too large (61) at 47 0
[h264 @ 0x1505ee790] error while decoding MB 47 0
[h264 @ 0x15059ca40] negative number of zero coeffs at 6 3
[h264 @ 0x15059ca40] error while de

0: 384x640 (no detections), 67.4ms
Speed: 3.5ms preprocess, 67.4ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 44.3ms
Speed: 0.7ms preprocess, 44.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 45.2ms
Speed: 1.2ms preprocess, 45.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 51.3ms
Speed: 1.1ms preprocess, 51.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 42.5ms
Speed: 1.0ms preprocess, 42.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 40.7ms
Speed: 0.9ms preprocess, 40.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 41.7ms
Speed: 0.7ms preprocess, 41.7ms inference, 0.4m

[h264 @ 0x15059ca40] corrupted macroblock 8 3 (total_coeff=-1)
[h264 @ 0x15059ca40] error while decoding MB 8 3


...

0: 384x640 (no detections), 41.4ms
Speed: 0.9ms preprocess, 41.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 47.3ms
Speed: 0.9ms preprocess, 47.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 40.0ms
Speed: 0.8ms preprocess, 40.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 38.9ms
Speed: 1.0ms preprocess, 38.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 39.9ms
Speed: 0.9ms preprocess, 39.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 39.9ms
Speed: 0.8ms preprocess, 39.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
frame is not empty!

0: 384x640 (no detections), 39.6ms
Speed: 1.0ms preprocess, 39.6ms inference, 0.3m

[h264 @ 0x1505ee790] negative number of zero coeffs at 33 6
[h264 @ 0x1505ee790] error while decoding MB 33 6


frame is not empty!

0: 384x640 1 person, 1 cat, 1 bowl, 131.3ms
Speed: 34.4ms preprocess, 131.3ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)
msg_dict is not empty!
frame is not empty!

0: 384x640 1 person, 1 cat, 1 bowl, 58.8ms
Speed: 2.0ms preprocess, 58.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
msg_dict is not empty!
frame is not empty!

0: 384x640 1 person, 2 cats, 1 bowl, 55.8ms
Speed: 0.7ms preprocess, 55.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
msg_dict is not empty!
frame is not empty!

0: 384x640 1 person, 1 cat, 1 bowl, 55.7ms
Speed: 1.5ms preprocess, 55.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
msg_dict is not empty!
frame is not empty!

0: 384x640 1 person, 1 cat, 1 bowl, 54.1ms
Speed: 1.8ms preprocess, 54.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
msg_dict is not empty!
frame is not empty!

0: 384x640 1 person, 1 cat, 1 bowl, 59.4ms
Speed: 1.6m

[h264 @ 0x15059ca40] cbp too large (3199971767) at 29 11
[h264 @ 0x15059ca40] error while decoding MB 29 11


: 

In [ ]:
test_img = proj_path + '/test_img/test1.jpg'

file= f"{proj_path}/saved_img/{datetime.datetime.now().strftime('%c')}.jpg"

img = cv2.imread(test_img)
cv2.imwrite(file, img)

if img is None:
  raise ValueError("Image doesn't exist.")
else:
  model = YOLO('yolov8n.pt')
  msg_dict = extract_model_prediction(model, img)
  if msg_dict is not None:
    push_ntfy(host, topic, msg_dict, file)